## Diff viewer

In [ ]:
import importlib
if importlib.util.find_spec("google.colab") is not None:
    !pip install datasets pandas ipywidgets

In [1]:
import difflib
from collections import defaultdict

import datasets
import ipywidgets as widgets
import pandas as pd
from IPython.display import display, HTML, Markdown

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/Sprakbanken/hugin-munin-error-labelling/main/data/behandlet/mistaken_lines.csv")
dataset = datasets.load_dataset("Teklia/NorHand-v3-line", split="test")

In [3]:
text_to_image = defaultdict(list)
for row in dataset:
    text_to_image[row["text"]].append(row["image"])
text_to_image = dict(text_to_image)

In [4]:
# Create the dropdown widget
options = df["arkindex_line_id"].tolist()
dropdown = widgets.Dropdown(options=options, value=options[0])

# Create the buttons
button_back = widgets.Button(description='Forrige')
button_forward = widgets.Button(description='Neste')

# Define the button click event handlers
def on_button_back_clicked(b):
    current_index = options.index(dropdown.value)
    if current_index > 0:
        dropdown.value = options[current_index - 1]

def on_button_forward_clicked(b):
    current_index = options.index(dropdown.value)
    if current_index < len(options) - 1:
        dropdown.value = options[current_index + 1]

# Attach the event handlers to the buttons
button_back.on_click(on_button_back_clicked)
button_forward.on_click(on_button_forward_clicked)

# Create an HBox to hold the widgets
hbox = widgets.HBox([button_back, dropdown, button_forward])

@widgets.interact(line=dropdown)
def show_diff(line):
    line = df.query("arkindex_line_id == @line", local_dict={"line": line}).squeeze()

    display(HTML(f"<a href='{line.highlight_url}'>ArkIndex</a> | <a href='{line.callico_task_url}'>Callico<a>" ))
    display(HTML(difflib.HtmlDiff().make_file(
        [line["ground_truth"]],
        [line["pylaia_prediction"]],
        "ground_truth",
        "pylaia_prediction"
    )))

    imgs = text_to_image.get(line["ground_truth"], [])
    for img in imgs:
        display(img)
    if not imgs:
        display(Markdown("**No line image found**"))

display(hbox)

interactive(children=(Dropdown(description='line', options=('e7ddbd5d-37e8-439c-aff3-779cea3ad95d', '0a0b69c6-…